
# Ethanol Concentration – Raman Tutorial

> _A step-by-step, **tutorial-style** notebook with helper functions and **synthetic example data**._
>
> **Goal:** Build a calibration curve for ethanol concentration in water using Raman spectra.
>
> **Important:** This is **not** a model solution for any specific lab dataset. It focuses on method, structure, and reusable utilities so you can adapt it to your own measurements.
>
> **Notebook created:** 2025-10-09 20:43



## Learning objectives

By the end of this tutorial you will be able to:
- Generate **synthetic Raman spectra** for ethanol–water mixtures to test your pipeline.
- Apply **baseline correction** (asymmetric least squares), smoothing, and simple normalization.
- **Select and integrate** peaks in user-defined windows.
- Build a **calibration curve** (linear model) and estimate goodness-of-fit.
- Perform a **quick k-fold cross‑validation** and compute simple uncertainty metrics.
- Export intermediate results for later reuse in other notebooks.



## Requirements

This notebook uses only common scientific Python packages. If you're running on Binder or a standard scientific Python, you should be fine.

- `numpy`, `scipy`, `matplotlib`, `pandas`


In [ ]:

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

# Ensure plots display inline
# (On some systems this is automatic)
# %matplotlib inline
print("Loaded: numpy, pandas, matplotlib, scipy")



## Helper functions

We keep helpers in this notebook for clarity. Later you can move them to a shared module (e.g. `helper_peakfit_functions.py`) and import them in multiple notebooks.


In [ ]:

def gaussian(x, amp, cen, wid):
    """Simple Gaussian peak."""
    return amp * np.exp(-0.5 * ((x - cen) / wid)**2)


def simulate_ethanol_spectrum(x):
    """Return a synthetic 'pure ethanol' spectrum (fingerprint region only).

    References for real peak positions vary; here we choose representative peaks
    to illustrate the method (not a perfect physical model).
    """
    # Representative ethanol fingerprint peaks (cm^-1): ~883, 1047, 1095, 1454
    peaks = [
        (1.0, 883, 8),
        (0.8, 1047, 10),
        (0.9, 1095, 9),
        (0.6, 1454, 12),
    ]
    y = np.zeros_like(x, dtype=float)
    for amp, cen, wid in peaks:
        y += gaussian(x, amp, cen, wid)
    return y


def simulate_water_spectrum(x):
    """Return a synthetic 'pure water' spectrum in the same region.

    Water has weaker fingerprint features but a broad OH band at higher wavenumbers.
    For the 800–1600 cm^-1 region, we add a small broad contribution plus a weak bending band.
    """
    y = 0.15 * np.exp(-0.5 * ((x - 1640) / 80)**2)  # bending (outside region, mostly 0 here)
    y += 0.05 * np.exp(-0.5 * ((x - 1200) / 150)**2)  # very broad weak background
    return y


def simulate_mixture(x, ethanol_fraction, baseline_curve='poly', noise_level=0.02, random_state=None):
    rng = np.random.default_rng(random_state)

    pure_eth = simulate_ethanol_spectrum(x)
    pure_h2o = simulate_water_spectrum(x)

    signal = ethanol_fraction * pure_eth + (1 - ethanol_fraction) * pure_h2o

    # 🔧 Baseline deutlich kleiner machen
    if baseline_curve == 'poly':
        b = 0.05 * ((x - x.mean()) / (x.ptp()/2))**2  # max. etwa 0.05
    elif baseline_curve == 'exp':
        b = 0.05 * np.exp(-(x - x.min()) / 400)
    else:
        b = 0.0

    # Leichte zufällige Skalierung
    scale = 0.9 + 0.2 * rng.random()
    y = scale * (signal + b)

    # Rauschen
    y += rng.normal(0, noise_level, size=x.size)
    return y, scale


In [ ]:

def baseline_asls(y, lam=1e5, p=0.01, niter=10):
    """Asymmetric Least Squares baseline correction.

    Eilers & Boelens (2005), implementation adapted for clarity.
    """
    import scipy.sparse as sp
    from scipy.sparse.linalg import spsolve

    L = len(y)
    D = sp.diags([1, -2, 1], [0, -1, -2], shape=(L, L-2))
    w = np.ones(L)
    for _ in range(niter):
        W = sp.diags(w, 0, shape=(L, L))
        Z = W + lam * D.dot(D.T)
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z


def preprocess(y, lam=1e5, p=0.01, niter=10, smooth_window=11, polyorder=2, norm='area'):
    """Baseline-correct, smooth, and normalize a spectrum.

    Returns the processed spectrum and the estimated baseline.
    """
    z = baseline_asls(y, lam=lam, p=p, niter=niter)
    y_corr = y - z
    if smooth_window is not None and smooth_window > 3:
        y_corr = savgol_filter(y_corr, smooth_window, polyorder)
    if norm == 'area':
        area = np.trapz(np.clip(y_corr, 0, None))
        if area != 0:
            y_corr = y_corr / area
    elif norm == 'max':
        m = np.max(np.abs(y_corr))
        if m != 0:
            y_corr = y_corr / m
    return y_corr, z


In [ ]:

def integrate_window(x, y, wmin, wmax):
    """Integrate y over [wmin, wmax] using trapezoidal rule."""
    mask = (x >= wmin) & (x <= wmax)
    if not np.any(mask):
        return 0.0
    return np.trapz(y[mask], x[mask])


def fit_gaussian_peak(x, y, wmin, wmax, p0=None):
    """Fit a single Gaussian to data in [wmin, wmax]. Returns popt, pcov."""
    mask = (x >= wmin) & (x <= wmax)
    xw, yw = x[mask], y[mask]
    if p0 is None:
        amp0 = yw.max() - yw.min()
        cen0 = xw[np.argmax(yw)]
        wid0 = (wmax - wmin)/6
        p0 = [amp0, cen0, wid0]
    popt, pcov = curve_fit(gaussian, xw, yw, p0=p0, maxfev=5000)
    return popt, pcov



## Generate a synthetic dataset

We'll simulate an **800–1600 cm⁻¹** region, with ethanol fractions from 0 to 1.  
Adjust ranges, noise, and baseline types to stress‑test the pipeline.


In [ ]:

# Wavenumber axis
x = np.linspace(800, 1600, 1601)

# Choose 'ground truth' ethanol fractions for calibration samples
fractions = np.linspace(0.0, 1.0, 11)  # 0%, 10%, ..., 100%
noise_level = 0.02

records = []
Y = []
for i, f in enumerate(fractions):
    y, scale = simulate_mixture(x, f, baseline_curve='poly', noise_level=noise_level, random_state=42+i)
    Y.append(y)
    records.append({'sample_id': i, 'ethanol_fraction': f, 'scale_factor': scale})

meta = pd.DataFrame.from_records(records)
spectra = pd.DataFrame(Y, columns=x).assign(sample_id=np.arange(len(fractions))).set_index('sample_id')
print(meta.head())
print("Spectra shape:", spectra.shape)

from pathlib import Path

# Prefer /mnt/data if available (used by Binder environment),
# otherwise create a local "data" folder next to the notebook
OUT_DIR = Path("/mnt/data") if Path("/mnt/data").exists() else Path("./data")
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("Saving files to:", OUT_DIR.resolve())

# ==============================================================
# Save synthetic spectra and metadata for reuse
# ==============================================================

out_csv = OUT_DIR / "synthetic_ethanol_raman.csv"
meta_csv = OUT_DIR / "synthetic_ethanol_meta.csv"

spectra.to_csv(out_csv)
meta.to_csv(meta_csv, index=False)

print("Saved synthetic data to:")
print("  •", out_csv)
print("  •", meta_csv)




### Quick look at the raw simulated spectra


In [ ]:

plt.figure(figsize=(8,4))
for i in spectra.index:
    plt.plot(spectra.columns.astype(float), spectra.loc[i].values, alpha=0.6)
plt.xlabel("Wavenumber (cm$^{-1}$)")
plt.ylabel("Intensity (a.u.)")
plt.title("Raw synthetic spectra (varied ethanol fraction)")
plt.tight_layout()
plt.show()



## Preprocess spectra (baseline → smooth → normalize)


In [ ]:

proc = {}
baselines = {}
for i in spectra.index:
    y = spectra.loc[i].values
    # vorher: y_corr, z = preprocess(y, ..., norm='area')
    y_corr, z = preprocess(y, lam=2e5, p=0.01, niter=12,
                           smooth_window=11, polyorder=2,
                           norm=None)   # << keine Normalisierung
    proc[i] = y_corr
    baselines[i] = z

proc_df = pd.DataFrame(proc).T
proc_df.columns = spectra.columns.astype(float)
print("Processed spectra shape:", proc_df.shape)


In [ ]:

# Visualize a couple of examples before/after
sample_ids = [0, 5, 10]  # 0%, 50%, 100%
fig, axes = plt.subplots(len(sample_ids), 1, figsize=(8, 6), sharex=True)
for ax, sid in zip(axes, sample_ids):
    x_axis = proc_df.columns.values
    y_raw = spectra.loc[sid].values
    y_base = baselines[sid]
    y_proc = proc_df.loc[sid].values
    ax.plot(x_axis, y_raw, label="raw", alpha=0.6)
    ax.plot(x_axis, y_base, label="baseline", alpha=0.8)
    ax.plot(x_axis, y_proc, label="processed", alpha=0.9)
    ax.set_title(f"Sample {sid}")
axes[-1].set_xlabel("Wavenumber (cm$^{-1}$)")
for ax in axes:
    ax.set_ylabel("Intensity (a.u.)")
    ax.legend()
fig.tight_layout()
plt.show()



## Choose peak windows (features)

Select one or two ethanol-dominated peaks and, optionally, a reference window.  
Here we demonstrate two approaches:
1. **Single-peak integration** (e.g., near **1047 cm⁻¹**).
2. **Ratio** of ethanol peak to a reference region to reduce day-to-day scaling effects.

> In your own data, inspect spectra to choose windows that are **well-resolved** and **robust**.


In [ ]:

# Define windows (cm^-1). Adjust to your spectrometer.
eth_peak = (1030, 1065)   # around 1047 cm^-1
ref_peak = (1300, 1350)     # around 883 cm^-1 (can be used as secondary ethanol marker or reference)

def extract_features(x, y):
    A_eth = integrate_window(x, y, *eth_peak)
    A_ref = integrate_window(x, y, *ref_peak)
    ratio = A_eth / A_ref if A_ref != 0 else np.nan
    return A_eth, A_ref, ratio

features = []
for i in proc_df.index:
    y = proc_df.loc[i].values
    A_eth, A_ref, ratio = extract_features(proc_df.columns.values, y)
    features.append({'sample_id': i, 'A_eth': A_eth, 'A_ref': A_ref, 'ratio_eth_ref': ratio})

feat_df = pd.DataFrame(features).merge(meta[['sample_id','ethanol_fraction']], on='sample_id')
feat_df.head()



### Inspect feature vs. concentration


In [ ]:

fig = plt.figure(figsize=(7,4))
plt.scatter(feat_df["ethanol_fraction"], feat_df["A_eth"])
plt.xlabel("Ethanol fraction (0–1)")
plt.ylabel("Integrated area (ethanol window)")
plt.title("Feature trend")
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(7,4))
plt.scatter(feat_df["ethanol_fraction"], feat_df["ratio_eth_ref"])
plt.xlabel("Ethanol fraction (0–1)")
plt.ylabel("Area ratio (ethanol/ref)")
plt.title("Ratio trend")
plt.tight_layout()
plt.show()



## Build a simple calibration model

We fit a **linear model**:  
\( y = a x + b \), where \(x\) is the ethanol fraction and \(y\) is the selected feature (area or ratio).

We also compute \(R^2\) and do a quick **k-fold cross-validation** for a sanity check.


In [ ]:

def linear_fit(x, y):
    # y = a*x + b
    a, b = np.polyfit(x, y, 1)
    yhat = a * x + b
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = 1 - ss_res/ss_tot if ss_tot != 0 else np.nan
    return a, b, r2


def kfold_cv(x, y, k=5, rng=0):
    rng = np.random.default_rng(rng)
    idx = np.arange(len(x))
    rng.shuffle(idx)
    folds = np.array_split(idx, k)
    r2s = []
    for i in range(k):
        test_idx = folds[i]
        train_idx = np.setdiff1d(idx, test_idx)
        a, b, _ = linear_fit(x[train_idx], y[train_idx])
        y_pred = a * x[test_idx] + b
        ss_res = np.sum((y[test_idx] - y_pred)**2)
        ss_tot = np.sum((y[test_idx] - np.mean(y[test_idx]))**2)
        r2 = 1 - ss_res/ss_tot if ss_tot != 0 else np.nan
        r2s.append(r2)
    return np.array(r2s)


x_cal = feat_df["ethanol_fraction"].values
y_cal = feat_df["A_eth"].values

a, b, r2 = linear_fit(x_cal, y_cal)
r2s = kfold_cv(x_cal, y_cal, k=5, rng=123)

print(f"a = {a:.4f}, b = {b:.4f}, R^2 = {r2:.4f}")
print("CV R^2 (5-fold):", np.round(r2s, 4), " -> mean:", np.nanmean(r2s).round(4))


In [ ]:

# Plot calibration
x_line = np.linspace(0, 1, 100)
y_line = a * x_line + b

plt.figure(figsize=(7,4))
plt.scatter(x_cal, y_cal, label="samples")
plt.plot(x_line, y_line, label="linear fit")
plt.xlabel("Ethanol fraction (0–1)")
plt.ylabel("Feature (area in ethanol window)")
plt.title("Calibration curve")
plt.legend()
plt.tight_layout()
plt.show()



### Alternative: use a ratio feature

Sometimes a ratio against a reference region is more robust to absolute intensity drift.


In [ ]:

x_cal2 = feat_df["ethanol_fraction"].values
y_cal2 = feat_df["ratio_eth_ref"].values

mask = np.isfinite(y_cal2)
a2, b2, r2_2 = linear_fit(x_cal2[mask], y_cal2[mask])
r2s2 = kfold_cv(x_cal2[mask], y_cal2[mask], k=5, rng=123)
print(f"[Ratio] a = {a2:.4f}, b = {b2:.4f}, R^2 = {r2_2:.4f}")
print("[Ratio] CV R^2:", np.round(r2s2, 4), " -> mean:", np.nanmean(r2s2).round(4))

x_line = np.linspace(0, 1, 100)
y_line = a2 * x_line + b2

plt.figure(figsize=(7,4))
plt.scatter(x_cal2, y_cal2, label="samples")
plt.plot(x_line, y_line, label="linear fit")
plt.xlabel("Ethanol fraction (0–1)")
plt.ylabel("Area ratio (ethanol/ref)")
plt.title("Calibration curve (ratio feature)")
plt.legend()
plt.tight_layout()
plt.show()



## Predicting concentration for a new spectrum

To test the pipeline, we simulate a new spectrum with an *unknown* ethanol fraction and run the same preprocessing and feature extraction, then invert the model.


In [ ]:

true_fraction = 0.37
y_new, _ = simulate_mixture(x, true_fraction, baseline_curve='exp', noise_level=0.02, random_state=999)
y_new_proc, _ = preprocess(y_new, lam=2e5, p=0.01, niter=12, smooth_window=11, polyorder=2, norm='area')
A_eth, A_ref, ratio = extract_features(x, y_new_proc)

# Invert linear model: y = a x + b  ->  x = (y - b)/a
est_fraction = (A_eth - b)/a if a != 0 else np.nan
est_fraction_ratio = (ratio - b2)/a2 if a2 != 0 else np.nan

print(f"True fraction: {true_fraction:.2f}")
print(f"Estimate (area): {est_fraction:.2f}")
print(f"Estimate (ratio): {est_fraction_ratio:.2f}")



### Simple uncertainty estimate (repeatability)

We can perturb the new spectrum with noise multiple times and see how the predicted concentration varies.  
This gives a **repeatability**-style uncertainty for the current pipeline.


In [ ]:

def predict_from_spectrum(y, model=('area', (a, b))):
    kind, params = model
    y_proc, _ = preprocess(y, lam=2e5, p=0.01, niter=12, smooth_window=11, polyorder=2, norm='area')
    if kind == 'area':
        A_eth, _, _ = extract_features(x, y_proc)
        A = A_eth
        aa, bb = params
        return (A - bb)/aa if aa != 0 else np.nan
    else:
        _, _, r = extract_features(x, y_proc)
        aa, bb = params
        return (r - bb)/aa if aa != 0 else np.nan

# Monte Carlo repeatability
rng = np.random.default_rng(123)
estimates = []
for _ in range(100):
    y_mc = y_new + rng.normal(0, 0.01, size=y_new.size)  # smaller noise than original
    estimates.append(predict_from_spectrum(y_mc, model=('area', (a, b))))
estimates = np.array(estimates)
print("Mean estimate:", np.nanmean(estimates).round(3))
print("Std deviation:", np.nanstd(estimates, ddof=1).round(5))



## Export features and models

You can export the features and fitted parameters to disk for reuse in other notebooks (e.g., `raman_spectra_analysis.ipynb`).


In [ ]:
OUT_DIR = Path("/mnt/data") if Path("/mnt/data").exists() else Path("./data")
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("Saving files to:", OUT_DIR.resolve())

# --------------------------------------------------------------
# Save features
# --------------------------------------------------------------
feat_path = OUT_DIR / "ethanol_features.csv"
feat_df.to_csv(feat_path, index=False)

# --------------------------------------------------------------
# Save fitted model parameters
# --------------------------------------------------------------
model_path = OUT_DIR / "ethanol_model_linear.txt"
with open(model_path, "w") as f:
    f.write("Model: y = a*x + b (area in ethanol window)\n")
    f.write(f"a = {a}\n")
    f.write(f"b = {b}\n")
    f.write(f"R2 = {r2}\n")

print("Saved feature table and model parameters to:")
print("  •", feat_path)
print("  •", model_path)



## Next steps & ideas

- Replace synthetic data with **your own spectra** and re-run.
- Try **different windows** (e.g., 880–900, 1085–1105, 1440–1470 cm⁻¹), or use **peak fitting** instead of window integration.
- Compare **normalization** strategies (area vs. max vs. internal standard).
- Try a **multi-feature regression** (e.g., multiple peak areas) or **PLS regression** for more complex mixtures.
- Quantify **LOD/LOQ** from baseline noise and calibration residuals.
- Log all metadata (laser power, integration time, grating, temperature) and inspect **systematic effects**.
